In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cv2
import cshow as cs
import scipy as sp

In [ ]:
cm = cs.colormapping.Colormap2d("rainbow_sixtyfive_dark_map_LUT")

In [ ]:
#Function to generate a grid of coordinates in numpy
#Resulting grid is indexed [coordinate, (x or y)][x grid position][y grid position]
def GenerateEqualSpaced2dGridCoordinates(grid_size,grid_dimension):
    #Single axis coordinate ticks, we only use square grids, just an ascending list
    GeneratingAxis = np.array(list(range(0,grid_size)))
    
    #Build an array of appropriate size, square of grid points, each point has 2! values, its x coordinate, and y coordinate
    CoordinateField = np.zeros((2,GeneratingAxis.size, GeneratingAxis.size))
    
    #Iterate each gridpoint, and fill in its position values
    for cx, iy, ix in np.ndindex(CoordinateField.shape):
        CoordinateField[0][ix][iy] = (ix - grid_size // 2) * grid_dimension / grid_size#Write x coordinate
        CoordinateField[1][ix][iy] = (iy - grid_size // 2) * grid_dimension / grid_size#Write y coordinate
        
    #Return the grid
    return CoordinateField


In [ ]:
B = 10
D = 8
M = 100
f1 = (D*M + B)/M
f2 = B/(M-1)

In [ ]:
mm = 10**-3
nm =  10 **-9
GridSize = 512
GridDimension = 0.1
BeamDimension = 1 * mm
freq_spacing = 2*np.pi*GridSize/GridDimension
lamb = 532 * nm
#Generate an example grid, GridsizexGridsize grid points
gridcoords = GenerateEqualSpaced2dGridCoordinates(GridSize,GridDimension)

In [ ]:
#Find the distance of each grid coordinate to a set location c, if c is not specified, just choose the center
#of the provided grid
def GridCoordinatesToDistances(x,c = "NaN"):
    if c == "NaN":
        #c not provided by user, just pick the center of the grid
        c = (x[0][x.shape[1] // 2][0] , x[1][0][x.shape[2] // 2])
    xp = x[0] - c[0]
    yp = x[1] - c[1]
    #D = v((x-c_x)^2 + (y-c_y)^2) 
    #Apply the distance formula on all grid points
    return np.sqrt(np.sum(np.square(np.array([xp,yp])),axis = 0))

In [ ]:
#Generate example distances
distances = GridCoordinatesToDistances(gridcoords)

In [ ]:
#Take in some grid coordinates, and output a 2d gaussian, whos square integral is one, and whose width is sigma
#(same idea as normalization in quantum mechanics)

def SpiderAperture(Coordinates,RInner, ROuter, ArmCount, ArmThickness):
    aperture = np.zeros((Coordinates.shape[1],Coordinates.shape[2])) + 1.0
    distances = GridCoordinatesToDistances(Coordinates)
    aperture[distances > ROuter] = 0.0
    aperture[distances < RInner] = 0.0
    arm0 =  np.zeros((Coordinates.shape[1],Coordinates.shape[2])) 
    arm0[np.logical_and(Coordinates[1] < 0, np.logical_and(Coordinates[0] < ArmThickness /2, Coordinates[0] > -ArmThickness /2))] = 1.0
    aperture[arm0 > 0.5] = 0.0
    
    for i in range(1, ArmCount):  
        armi = sp.ndimage.rotate(arm0,360/ArmCount * (i),reshape=False)
        aperture[armi > 0.5] = 0.0
    return aperture
    

In [ ]:
plt.imshow(SpiderAperture(gridcoords,0.02,0.02,5,0.005))
plt.colorbar()

In [ ]:
def NormalizedGaussian(coordinates, sigma):
    #The gaussian falls as the distance to its center squared, with rate scaling by sigma, the distribution width
    unnormalized_gaussian = np.exp(-(GridCoordinatesToDistances(coordinates)/sigma)**2)
    
    #Return the gaussian, but normalized
    return unnormalized_gaussian/np.sqrt(np.sum(np.square(unnormalized_gaussian)))
    

In [ ]:
amplitude = np.zeros(gridcoords[0].shape) + 1j#
#amplitude = NormalizedGaussian(gridcoords,0.05) * 1.0j
amplitude/= np.sqrt(np.sum(np.square(np.abs(amplitude))))
aperture = SpiderAperture(gridcoords,0.004,0.02,3,0.002)

amplitude *= aperture

amplitude /= np.sqrt(np.sum(np.abs(amplitude)))
tilt = 0.002 * np.pi/180
amplitude += amplitude * np.exp(2*np.pi * 1.0j*gridcoords[0]* tilt / lamb)

In [ ]:
plt.title("Amplitude after aperture")
plt.imshow(cm(amplitude))
cs.colordisk(cm, insetdim = 0.16)
plt.show()

In [ ]:
print(np.sum(np.square(np.abs(amplitude))))

In [ ]:
random_phasing = np.zeros(gridcoords[0].shape)
import random
import time
random.seed(time.time())
for gaussx in range(2):
    for gaussy in range(2):
        random_phasing +=1 * random.random() * np.exp(-np.square(np.square(GridCoordinatesToDistances(gridcoords, c = (GridSize/2 * gaussx*random.random() * GridDimension/GridSize ,GridSize/2 * gaussy *random.random()* GridDimension/GridSize - GridDimension/8) )))/np.square(GridDimension / 14)/np.square(GridDimension / 14))
plt.imshow(random_phasing)
plt.colorbar()
plt.show()

In [ ]:
#amplitude *= np.exp(1.0j * random_phasing)
amplitude *= np.exp(-1.0j * np.pi/lamb/(f1) * distances**2)
plt.title("Amplitude after rain drops")
plt.imshow(cm(amplitude))
cs.colordisk(cm, insetdim = 0.16)
plt.show()
out = amplitude

outx = np.sum(np.square(np.abs(out)), axis = 1)/np.sum(np.square(np.abs(out)))

outy = np.sum(np.square(np.abs(out)), axis = 0)/np.sum(np.square(np.abs(out)))
#print(np.sum(np.abs(np.square(out))))


#PlotAmplitudeIntensity(out)
#plt.show()
#plt.plot(outx)
#c0 = sp.stats.moment(np.square(np.abs(out)), moment=1, axis=0, nan_policy='propagate')
#c1 = sp.stats.moment(np.square(np.abs(out)), moment=1, axis=1, nan_policy='propagate' )
np.sum(outx*gridcoords[0,:,0])/np.sum(outx)
sf = 1
newc = (gridcoords[0,:,0]) * sf
m1x = np.sum(outx*newc)
m1y = np.sum(outy*newc)

m2y = np.sum(outy*np.square(newc - m1y))
m2x = np.sum(outx*np.square(newc - m1x))

display(2*np.sqrt(m2x))

In [ ]:
z = D
#if(i % 10) == 0:
    #plt.title(str(z / mm))
    #PlotAmplitudePhase(np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances[:,:])))
    #plt.show()
FT_m = np.fft.fftshift(np.fft.fft2(amplitude * np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances[:,:]))),axes=(0,1))#np.roll(np.roll(np.fft.fft2(final_field * np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances[:,:]))),GridSize//2, axis = 1),GridSize//2,axis = 0)
#plt.imshow(np.abs(np.square(FT_m)))
#plt.title("FFT")
#plt.show()

#kernel = np.exp(1.0j*2*np.pi*z/lamb)/(1.0j*lamb*z)*np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances2[:,:]))
#out = cv2.filter2D(src=final_field,ddepth=-1,kernel = kernel)#kernel,mode = "same")/np.sqrt(np.sum(np.abs(np.square(out))))
sf = GridSize/(GridDimension**2)*lamb*z
newco = gridcoords * sf

out = FT_m * np.exp(1.0j /2 / z * 2 * np.pi/ lamb *np.square(GridCoordinatesToDistances(newco)))

In [ ]:
plt.title("Amplitude after prop")
plt.imshow(cm(out))
cs.colordisk(cm, insetdim = 0.16)
plt.show()

outx = np.sum(np.square(np.abs(out)), axis = 1)/np.sum(np.square(np.abs(out)))

outy = np.sum(np.square(np.abs(out)), axis = 0)/np.sum(np.square(np.abs(out)))
#print(np.sum(np.abs(np.square(out))))


#PlotAmplitudeIntensity(out)
#plt.show()
#plt.plot(outx)
#c0 = sp.stats.moment(np.square(np.abs(out)), moment=1, axis=0, nan_policy='propagate')
#c1 = sp.stats.moment(np.square(np.abs(out)), moment=1, axis=1, nan_policy='propagate' )
np.sum(outx*gridcoords[0,:,0])/np.sum(outx)
sf = GridSize/(GridDimension**2)*lamb*z
newc = (gridcoords[0,:,0]) * sf
m1x = np.sum(outx*newc)
m1y = np.sum(outy*newc)

m2y = np.sum(outy*np.square(newc - m1y))
m2x = np.sum(outx*np.square(newc - m1x))

display(2*np.sqrt(m2x))

In [ ]:
out *= np.exp(1.0j * np.pi/lamb/(f2) * GridCoordinatesToDistances(newco)**2)



z = B
#if(i % 10) == 0:
    #plt.title(str(z / mm))
    #PlotAmplitudePhase(np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances[:,:])))
    #plt.show()
FT_m = np.fft.fft2(out * np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(GridCoordinatesToDistances(newco))))#np.roll(np.roll(np.fft.fft2(final_field * np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances[:,:]))),GridSize//2, axis = 1),GridSize//2,axis = 0)
#plt.imshow(np.abs(np.square(FT_m)))
#plt.title("FFT")
#plt.show()

#kernel = np.exp(1.0j*2*np.pi*z/lamb)/(1.0j*lamb*z)*np.exp(1.0j*2*np.pi/lamb/(2*z)*np.square(distances2[:,:]))
#out = cv2.filter2D(src=final_field,ddepth=-1,kernel = kernel)#kernel,mode = "same")/np.sqrt(np.sum(np.abs(np.square(out))))
out2 = FT_m#* np.exp(1.0j /2 / z * 2 * np.pi/ lamb *np.square(distances))

In [ ]:
plt.title("Amplitude after prop 2")
plt.imshow(np.abs(np.square(out2)))
cs.colordisk(cm, insetdim = 0.16)
plt.show()

In [ ]:
h = np.exp(1.0j /2 / z * 2 * np.pi/ lamb *np.square(distances))
#out2 = sp.signal.convolve2d(out,h, mode = 'same')

In [ ]:
plt.title("Amplitude after prop 2")
plt.imshow(cm(out2))
cs.colordisk(cm, insetdim = 0.16)
plt.show()

In [ ]:
image += out2

In [ ]:
plt.title("Amplitude after prop 2")
plt.imshow(np.abs(np.square(image)))
cs.colordisk(cm, insetdim = 0.16)
plt.show()